# Taaltheorie & Taalverwerking 2017 - Assignment 1


The objective of this exercise is to implement a system that will allow us to execute and analyse any arbitrary finite state machine (FSM) specified in Python. Recall that an FSM is a tuple $\langle\Sigma,Q,q_0,F,\delta\rangle$, where $\Sigma$ is the alphabet, $Q$ is the set of states, $q_0$ is the initial state, and $F$ is the set of final states. We use a provided Python class to represent both states in $Q$ and symbols in the alphabet $\Sigma$, and we use Python strings to represent tokens/symbols. 


In [17]:
# FILL THIS IN FOR YOUR GROUP, also name your file as: tttv_ass1_<group>_<name1>_<name2>.ipynb
# Group        : G
# Name - UvaID : Deborah Lambregts - TODO
# Name - UvaID : Bram Otten - 10992456
# Date         : 09-04-2017

In [18]:
import copy

class FSM:

    def __init__(self):
        """Contructor of the FSM."""
        self._initial_state = None
        self._transitions = {}
        self._end_states = set()

    def get_transitions(self):
        """Returns a deep copy of the transitions of the current FSM."""
        return copy.deepcopy(self._transitions)

    def add_transition(self, from_state, token, to_state):
        """
        Add a transition to the FSM.
        Args:
            from_state: String representing of a state.
            token: A letter or word, depending on the context.
            to_state: String representation of a state.

        Returns:
            True/False depending on whether the transition could be added.
        """
        from_state = from_state.upper()
        to_state = to_state.upper()
        if token == "" and from_state == to_state:
            print((from_state, token, to_state))
            print("This would generate an infinite loop, transition has not been added.")
            return False
        # Create an empty set for new keys.
        if from_state not in self._transitions:
            self._transitions[from_state] = set()
        self._transitions[from_state].add((token, to_state))
        return True

    def remove_transition(self, from_state, token, to_state):
        """
        Args:
            from_state: String representing of a state.
            token: A letter or word, depending on the context.
            to_state: String representation of a state.

        Returns:
            True/False depending on whether the transition could be removed.
        """
        from_state = from_state.upper()
        to_state = to_state.upper()
        # Check if the transition exists
        if from_state in self._transitions:
            if (token, to_state) in self._transitions[from_state]:
                self._transitions[from_state].remove((token, to_state))
                print("Transition ({}, {}, {}) succesfully removed".format(from_state, token, to_state))
                return True
        print("Transition ({}, {}, {}) could not be found.".format(from_state, token, to_state))
        return False


    def get_initial_state(self):
        """Returns a deep copy of the transitions of the current FSM."""
        return copy.deepcopy(self._initial_state)

    def set_initial_state(self, state):
        """Sets the initial state of the FSM."""
        self._initial_state = state.upper()

    def get_end_states(self):
        """Returns a deep copy of the end states of the current FSM."""
        return copy.deepcopy(self._end_states)

    def add_end_state(self, state):
        """Add an end state to the FSM."""
        self._end_states.add(state.upper())

    def remove_end_state(self, state):
        """Remove an end state from the FSM if it exists."""
        if state.upper() in self._end_states:
            self._end_states.remove(state.upper())
            print("State {} succesfully removed as end state".format(state))
            return True
        else:
            print("State {}, is not one of the end states, and can not be removed.".format(state))
            return False

    def print_details(self):
        """Prints all the details of the current FSM."""
        print("FSM details:")
        print("- Initial state:")
        print("  - {}".format(self._initial_state))
        print("- End states:")
        end_states = list(self._end_states)
        end_states.sort()
        for state in end_states:
            print("  - {}".format(state))
        print("- Transitions: ")
        for state in self._transitions:
            print("  - {}".format(state))

![First FSM should be here, else add the image in the root of your jupyter notebook](fsm1.png)

We represent the transition function~$\delta$ as a Python dictionary in which for each start node, the transition tokens and the end nodes are stored. The following example shows some basic operations for adding and reading transitions.
You can do this yourself by using the **add\_transition(state1, token, state2)** function of the FSM object: 

In [19]:
fsm1 = FSM()
fsm1.add_transition('q0','a','q1')
fsm1.add_transition('q0','b','q1')
fsm1.add_transition('q1','b','q2')
fsm1.add_transition('q1','','q2')
fsm1.add_transition('q1','a','q2')
fsm1.print_details()

FSM details:
- Initial state:
  - None
- End states:
- Transitions: 
  - Q0
  - Q1


We furthermore use the function **fsm.set\_initial\_state()** to indicate the name of the initial state, and the predicate **fsm.add\_end\_state()** to specify any number of final states (in our example there happens to be just a single final state). For our example, this looks as follows:


In [20]:
fsm1.set_initial_state('q0')
fsm1.add_end_state('q2')
fsm1.print_details()

FSM details:
- Initial state:
  - Q0
- End states:
  - Q2
- Transitions: 
  - Q0
  - Q1


Inside the FSM class/object the variables such as **self.\_initial\_state** are accessed directly, if you want to access these from outside the object, you should use the **get** functions:


In [21]:
# Accessing the internal variables of the FSM object correctly, from the outside
print("- Initial state:")
print("  - {}".format(fsm1.get_initial_state()))
# End states are stored as a set(), resulting in only unique values.
print("- End states:")
end_states = list(fsm1.get_end_states())
for state in end_states:
  print("  - {}".format(state))
# transitions = dictionary of tuples
print("- Transitions:")
transitions = fsm1.get_transitions()
for state in transitions:
  for (token, end_state) in transitions[state]:
    print("  - {} -> {} -> {}".format(state, token, end_state))

- Initial state:
  - Q0
- End states:
  - Q2
- Transitions:
  - Q0 -> b -> Q1
  - Q0 -> a -> Q1
  - Q1 -> a -> Q2
  - Q1 -> b -> Q2
  - Q1 ->  -> Q2


Now answer the following questions:

### Question 1 (3 pts)

In class you have learned to always specify the alphabet~$\Sigma$ and the set of states~$Q$ when defining an FSM. This is important for clarity, but of course this information in principle can also be extracted from the specification of $\delta$ (only states and symbols not involved in any transition cannot be recovered this way).

Write a function **states(fsm)** to obtain the list of all states used anywhere in the provided FSM object (i.e., all states showing that you have provided via: **fsm.set\_initial()**, **fsm.add\_end\_state()**, and **fsm.add\_transition()**). Your list should be sorted alphabetically and not contain any duplicates. Example:




In [22]:
def states(fsm):
    all_states = []
    transitions = fsm.get_transitions()
    
    for initial_state in transitions:
        all_states.append(initial_state)
        
        for (_, end_state) in transitions[initial_state]:
            all_states.append(end_state)
    
    output_list = list(set(all_states))
    output_list.sort()

    return output_list

# Output for fsm1 should be ['q0', 'q1', 'q2']
states(fsm1)

['Q0', 'Q1', 'Q2']

Now write a function **alphabet(fsm)** to recover all the alphabet symbols used anywhere in the current FSM specification. Again, the list returned should be sorted and free of duplicates. Example:


In [23]:
def alphabet(fsm):
    
    all_transitions = []
    transitions = fsm.get_transitions()
    
    for state in transitions:
        for (element, _) in transitions[state]:
            all_transitions.append(element)
    
    output_list = list(set(all_transitions))
    output_list = list(filter(None, output_list))
    output_list.sort()
    
    return output_list

# Output for fsm1 should be ['a', 'b']
alphabet(fsm1)

['a', 'b']

### Question 2 (4 pts)

Implement a function **accept(fsm, string)** to check whether a given string, is accepted by the given FSM.

__Hints:__ A breadth-first algorithm, can work via a queue (just look it up), python has a queue class, called *deque*, from *from collections import deque*. On another note, you can also try recursion in python.
Also make sure you don't forget about the $\epsilon$-transitions!


In [73]:
from collections import deque

### DOESN'T WORK !!!
def accept(fsm, query_string):
    
    index = 0
    past = set()
    future = deque([fsm.get_initial_state()])
    transitions = fsm.get_transitions()
    
    while future:
        
        testing = future.popleft()
        print(testing)
        ### DOESN'T WORK !!!
        
        for (token, end) in transitions[testing]:
            if query_string[index] == token:
                if end not in past:
                    past.add(end)
                    future.append(end)
                
        index += 1     
        print(future)
        
        # The end.
        if index == len(query_string):
            if testing in fsm.get_end_states():
                return True;
            return False;
        ### DOESN'T WORK !!!
    
    return False

print(accept(fsm1, "b"))   # True
print(accept(fsm1, "aab")) # False

Q0
deque(['Q1'])
False
Q0
deque(['Q1'])
Q1
deque(['Q2'])
Q2


KeyError: 'Q2'

Unlike Prolog, python does not have enforced backtracking (repeatedly pressing the semicolon key) to generate all strings accepted by the current FSM currently in memory, but for **fsm1** we know that only the following strings should be accepted:


In [25]:
print(accept(fsm1, "ab"))
print(accept(fsm1, "aa"))
print(accept(fsm1, "a"))
print(accept(fsm1, "bb"))
print(accept(fsm1, "ba"))
print(accept(fsm1, "b"))

False
False
False
False
False
False


### Question 3 (4 pts)

Now create a new FSM object, called **fsm2** and write up a specification of the following FSM:


![Second FSM should be here, else add the image in the root of your jupyter notebook](fsm2.png)

In [26]:
fsm2 = FSM()
fsm2.add_transition('q0', 'x', 'q1')
fsm2.add_transition('q1', 'x', 'q1')
fsm2.add_transition('q1', 'y', 'q2')
fsm2.add_transition('q2', 'x', 'q1')
fsm2.set_initial_state('q0')
fsm2.add_end_state('q1')
fsm2.print_details()

FSM details:
- Initial state:
  - Q0
- End states:
  - Q1
- Transitions: 
  - Q0
  - Q1
  - Q2


If you have done everything right, then you should be able to verify, for instance, that <u>xyxx</u> is a string that is accepted by this FSM:


In [27]:
print(accept(fsm2, "xyxx")) # True

False


Observe that this new FSM accepts an *infinite* number of strings. In which order it will enumerate them depends on the exact order in which you specify your transitions.
As this FSM can generate an infinite number of strings, we can not just test a small list to see if it works correctly. If we want to make sure not to miss any accepted strings, we could try to search the space of all accepted strings in the order of their length.

To this end, implement a function **list_accepted(fsm, length)** to collect the list of all strings of a given length accepted by the given FSM. This list should be sorted and not contain any duplicates. Example:


In [28]:
def list_accepted(fsm, length):
    output_list = ["TODO"]
    return output_list

print(list_accepted(fsm2, 4)) # ["xxxx", "xxyx", "xyxx"]

['TODO']


How many distinct strings of length 10 are accepted by our (new) FSM? How many of length 20? Document how you have obtained the answers to these questions.
<h4>Answers:</h4>

In [29]:
# The first and last letters have to be x'es.

# There can be at most (n-2)/2 y's in betweeon those letters.
# The y's can be shuffled around in the n-2 spots, in ((n-2)/2)! ways. 
# 24 with n=10, 362880 with n=20.

### Question 4 (4 pts)
Implement a function **deterministic(fsm)** to determine whether the FSM currently in memory is deterministic. This query should fail for our first FSM and succeed for the second:

In [30]:
def deterministic(fsm):
    
    combos = {}
    transitions = fsm.get_transitions()
    
    for state in transitions:
        combos[state] = []
        for (token, end_state) in transitions[state]:
            if token == "" or token in combos[state]:
                return False
            
            combos[state] += token
    
    return True

print(deterministic(fsm1)) # False
print(deterministic(fsm2)) # True

False
True


### Question 5 (3 pts)
Now create a new FSM object, called **fsm3** and write up a specification of an FSM that defines the present tense inflectional paradigm of the English verb "do".

If you have done this right, only the frist three strings should be considered correct by **accept(fsm3, string)**:

In [76]:
fsm3 = FSM()

fsm3 = FSM()
fsm3.add_transition('q0', 'd', 'q1')
fsm3.add_transition('q1', 'o', 'q2')

fsm3.add_transition('q2', 'e', 'q3')
fsm3.add_transition('q3', 's', 'q2')

fsm3.add_transition('q2', 'i', 'q3')
fsm3.add_transition('q3', 'n', 'q4')
fsm3.add_transition('q4', 'g', 'q2')

fsm3.set_initial_state('q0')
fsm3.add_end_state('q2')
fsm3.print_details()

print(accept(fsm3, "do")) # True
print(accept(fsm3, "does")) # True
print(accept(fsm3, "doing")) # True
print(accept(fsm3, "dos")) # False
# TODO

FSM details:
- Initial state:
  - Q0
- End states:
  - Q2
- Transitions: 
  - Q0
  - Q1
  - Q2
  - Q3
  - Q4
Q0
deque(['Q1'])
Q1
deque(['Q2'])
False
Q0
deque(['Q1'])
Q1
deque(['Q2'])
Q2
deque(['Q3'])
Q3
deque([])
False
Q0
deque(['Q1'])
Q1
deque(['Q2'])
Q2
deque(['Q3'])
Q3
deque(['Q4'])
Q4
deque([])
False
Q0
deque(['Q1'])
Q1
deque(['Q2'])
Q2
deque([])
True


Is your FSM deterministic or nondeterministic?
<h4>Answers:</h4>

In [32]:
# Deterministic, as there is only one path a certain string can follow.